In [ ]:
import obspy
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from ipywidgets import interact, interact_manual
from taup_distance import taup_distance
from obspy.taup import TauPyModel
model = TauPyModel('model_data/model4taup10.npz')

st = obspy.read('event_data/S0235b/waveforms/waveforms_VBB.mseed')
st.filter('highpass', freq=1./10)

TT = np.load('helpers/traveltimes.npz')
#st.plot()

In [ ]:
def plot_waveform(t_P=100, t_S=200):
    fig = plt.figure(constrained_layout=True)

    gs = GridSpec(3, 3, figure=fig)
    ax_Z = fig.add_subplot(gs[0, 0:2])
    ax_N = fig.add_subplot(gs[1, 0:2], sharex=ax_Z)
    ax_E = fig.add_subplot(gs[2, 0:2], sharex=ax_Z)
    ax_dist = fig.add_subplot(gs[0, 2])

    for tr, ax in zip(st, (ax_Z, ax_N, ax_E)):
        ax.plot(tr.times(), tr.data)
        ax.set_xlim(100, 500)

    ax_Z.axvline(t_P, c='r')
    ax_N.axvline(t_S, c='r')
    ax_E.axvline(t_S, c='r')
    
    
    dist = taup_distance.get_dist(model=model, tSmP=t_S-t_P, depth=50)
    
    if dist is not None:
        t_P_theo = model.get_travel_times(distance_in_degree=dist, 
                                          source_depth_in_km=50, 
                                          phase_list=['P'])[0].time
        t_S_theo = model.get_travel_times(distance_in_degree=dist, 
                                          source_depth_in_km=50, 
                                          phase_list=['S'])[0].time

        for phase in ['P', 'S', 'PP', 'SS', 'ScS']:
            t = np.asarray(TT[phase])
            ax_dist.plot(t[:,0], t[:,1], label=phase)
        ax_dist.plot(dist, t_S_theo, 'o')
        ax_dist.plot(dist, t_P_theo, 'o')
        ax_dist.plot([dist, dist], [t_P_theo, t_S_theo], 'k')
    return ax_Z, ax_N, ax_E


interact(plot_waveform,
            t_P=(50, 1000),
            t_S=(50, 1000))